# zhsegment: default program

In [1]:
from zhsegment import *

## Run the default solution on dev

In [15]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签 订 高 科 技 合 作 协 议
新 华 社 上 海 八 月 三 十 一 日 电 （ 记 者 白 国 良 、 夏 儒 阁 ）
“ 中 美 合 作 高 科 技 项 目 签 字 仪 式 ” 今 天 在 上 海 举 行 。


## Evaluate the default output

In [6]:
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)


score: 0.27


# Documentation

## Implementation of Entry class

We defined a `class Entry` inside the `class Segment` that consists four components: `word`, `start_pos`, `log_prob`, `back_pointer` for iterating with the dynamic programming table `chart`.

Within the `class Entry`, we also implemented the `__eq__` method to compare instances of the class. If the mismatches the components or is not an isntance of the class, `__eq__` method will return false.

The `__lt__` and `__gt__` are also implemented for comparing the `log_prob` component. Since we want to push higher probability to the front of the heap, the operator signs were assigned reversely due to functionality of Python's heap queue algorithm.

In [ ]:
class Entry:
    def __init__(self,word,start_pos,log_prob,back_pointer):
        self.word = word
        self.start_pos = start_pos
        self.log_prob = log_prob
        self.back_pointer = back_pointer
    
    def __eq__(self,other):
        if (self.word == other.word and self.start_pos == other.start_pos and self.log_prob == other.log_prob):
            return True
        if (not isinstance(other, type(self))):
            return False
        else:
            False
        
    def __lt__(self,other):
        return self.log_prob > other.log_prob
        
    def __gt__(self,other):
        return self.log_prob <= other.log_prob

---
Implementation of Iterative Segmenter Algorithm
---
We added the combination of unigram and bigram iterative algorithm (`segment_iter`) to `class Segment`.


### Segmenter Algorithm with Unigram Probability

The code below is the initial iterative Segmenter algorithm in unigram approach. Considering probability for long unseen words, we performed additive smoothing on the unigram probability model by modifying `self.missingfn` in `class Pdist`, and the overall dev score of the unigram approach reached $0.93$. The code for unigram's `Pdist` is included at the bottom of the code segment, where `self.missingfn` is changed from:

```
self.missingfn = missingfn or (lambda k, N: 1./ N)
```
to
```
self.missingfn = missingfn or (lambda k, N: 1./(N*1100**len(k)))
``` 

In [ ]:
# Unigram approach
class Segment:

    def __init__(self, Pw):
        self.Pw = Pw

    def segment(self, text):
        ## Initialize the heap ##
        heap = []
        chart = {}
        # for each word that matches input at pos 0, insert entry into heap
        
        for i in range(len(text)):
            word = text[0:i+1]
            chart[i] = Entry(None, None, None, None)
            if word in self.Pw:
                heapq.heappush(heap, Entry(word, 0, log10(self.Pw(word)), None))    ## heapq.heappush pushes the entry item to the heap queue
        
        ## Iteratively fill in chart[i] for all i ##
        while heap:
            entry = heap[0]     # top entry in the heap
            endindex = entry.start_position + len(entry.word) - 1
            if chart[endindex].back_pointer is not None:        
                # preventry = chart[endindex]
                preventry = chart[endindex].back_pointer
                if entry.log_probability > preventry.log_probability:
                    chart[endindex] = entry
                else:
                    continue    ## we have already found a good segmentation until endindex ##
            else:
                chart[endindex] = entry
        
        for j in range(endindex+1, len(text)):
            newword = text[endindex+1 : j+1]
            if newword in self.Pw:
                newentry = Entry(newword, endindex+1, entry.log_probability+log10(self.Pw(newword), entry))
                if newentry not in heap:
                    heapq.heappush(heap, newentry)
        
        ## Get the best segmentation ##
        finalindex = len(text)
        finalentry = chart[finalindex]
        segmentation = []
        while finalentry:
            segmentation.append(finalentry.word)
            finalentry = finalentry.back_pointer

        return segmentation

    def Pwords(self, words): 
        "The Naive Bayes probability of a sequence of words."
        return product(self.Pw(w) for w in words)

class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./(N*1100**len(k)))
    def __call__(self, key): 
        if key in self: return (self[key]+1)/(self.N)  
        else: return self.missingfn(key, self.N)

### Segmenter Algorithm with Unigram and Bigram Probability

The code below is the initial iterative Segmenter algorithm combining unigram and bigram approach.

In an attempt to weight the predictive power of the unigram or bigram sequences being chosen, we implemented Jelinek-Mercer smoothing through the linear interpolation form. By varying the lambda parameter (the final one chosen was $\lambda = 0.9$) we obtained a score of $0.92$ on the `dev` dataset.

Considering probability for long unseen words, we performed Laplace smoothing on the unigram probability model by modifying `self.missingfn` in `class Pdist`, and the overall dev score of the unigram approach reached 0.92. The code for unigram's `Pdist` is included at the bottom of the code segment, where `self.missingfn` is changed from:

```
self.missingfn = missingfn or (lambda k, N: 1./ N)
```
to
```
self.missingfn = missingfn or (lambda k, N: (1+1)/(N*1000**len(k)))
``` 

In [4]:
class Segment:
    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        segmentation = [ w for w in text ] # segment each char into a word
        return segmentation

    def segment_iter(self, text):
        heap, chart = [], {}
        
        word = ""
        for w in range(0,len(text)):
            word += text[w]
            chart[w] = self.Entry(None,None,None,None)
            if (word in self.Pw_uni or len(word) < 6):
                heapq.heappush(heap, self.Entry(word, 0, 0.9*log10(self.Pw_uni(word)), None))
            
        
        while (len(heap)!=0):
            top_entry = heapq.heappop(heap)
            endIndex = top_entry.start_pos + len(top_entry.word)-1
            if (chart[endIndex].back_pointer is not None):
                prevEntry = chart[endIndex].back_pointer
                if(top_entry.log_prob > prevEntry.log_prob):
                    chart[endIndex] = top_entry
                else:
                    continue
            else:
                chart[endIndex] = top_entry

            newWord = ""
            for n in range(endIndex+1,len(text)):
                newWord += text[n]
                if (newWord in self.Pw_uni or len(newWord) < 6):
                    wordPair = top_entry.word+" "+newWord
                    if (wordPair in self.Pw_bi and top_entry.word in self.Pw_uni):
                        newEntry = self.Entry(newWord, endIndex+1, top_entry.log_prob + 0.1*log10(self.Pw_bi(wordPair)/self.Pw_uni(newWord)), top_entry)
                    else:
                        newEntry = self.Entry(newWord, endIndex+1, top_entry.log_prob+log10(self.Pw_uni(newWord)), top_entry)
                    if (newEntry not in heap):
                        heapq.heappush(heap, newEntry)
        
        finalIndex = len(text)-1
        finalEntry = chart[finalIndex]
        best_segment = []
        while (finalEntry is not None):
            best_segment.insert(0,finalEntry.word)
            finalEntry = finalEntry.back_pointer
        return best_segment

class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: (1+1)/(N*1000**len(k)))
    def __call__(self, key): 
        if key in self: return (self[key]+1)/(self.N)  
        else: return self.missingfn(key, self.N)

## Evaluate the final output

In [2]:
Pw_uni = Pdist(data=datafile("../data/count_1w.txt"))
Pw_bi = Pdist(data=datafile("../data/count_2w.txt"))
segmenter = Segment(Pw_uni,Pw_bi)
output_full = []
with open('../data/input/dev.txt') as f:
    for line in f:
        output = " ".join(segmenter.segment_iter(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白国良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


In [3]:
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.92


## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

We have approached multiple ways in implementing the unigram and bigram models. For unigram we first implemented the baseline algorithm, but ended up with `dev.score` of $0.88$ onl. So we further enhanced accuracy to $0.93$ by performing additive and laplace smoothing on the probability distribution of the counts of words in the count files.

We then moved on to implement the bigram model combining our initial unigram iterative algorithm, which gave us a `dev.score` of $0.81$. Then we attempted to improve accuracy by using backoff smoothing to prioritize unigram probabilities over grouped tokens, which boost the score to $0.90$. 

Eventually, we attempted in performing linear interpolation on the models, where we implemented Jelinek-Mercer smoothing by testing various lambda parameter. The final `dev.score` we got with JM smoothing is $0.92$.